In [1]:
import pandas as pd

# read
df = pd.read_parquet('01.parquet')


df.head()

,Ticker,Date/Time,Open,High,Low,Close,Volume,Open Interest
0,BANKNIFTY,2023-01-02 09:07:00,43038.25,43038.25,43038.25,43038.25,0.0,0.0
1,BANKNIFTY,2023-01-02 09:15:00,43079.10,43131.75,43036.00,43126.70,0.0,0.0
2,BANKNIFTY,2023-01-02 09:16:00,43134.00,43151.25,43097.20,43117.15,0.0,0.0
3,BANKNIFTY,2023-01-02 09:17:00,43118.30,43121.55,43071.55,43109.85,0.0,0.0
4,BANKNIFTY,2023-01-02 09:18:00,43105.70,43108.60,43061.40,43070.45,0.0,0.0
